## Silver Layer Code

In [0]:
# To Access data

spark.conf.set("fs.azure.account.auth.type.awdedatalake.dfs.core.windows.net", "SAS")
spark.conf.set("fs.azure.sas.token.provider.type.awdedatalake.dfs.core.windows.net", 
               "org.apache.hadoop.fs.azurebfs.sas.FixedSASTokenProvider")
spark.conf.set("fs.azure.sas.fixed.token.awdedatalake.dfs.core.windows.net", "sv=2024-11-04&ss=b&srt=sco&sp=rwdlacx&se=2025-05-07T16:04:32Z&st=2025-05-04T08:04:32Z&spr=https&sig=RXOct57bo036UbcRjJOWBMKOQV2F%2FO%2BaXfRGisYKlys%3D")

#### Reading Data

In [0]:
# Read Data

df_cal = spark.read.format('csv')\
              .option("inferSchema", "true")\
              .option("header", "true")\
              .load('abfss://bronze@awdedatalake.dfs.core.windows.net/AdventureWorks_Calendar')

In [0]:
df_cust = spark.read.format('csv')\
              .option("inferSchema", "true")\
              .option("header", "true")\
              .load('abfss://bronze@awdedatalake.dfs.core.windows.net/AdventureWorks_Customers')

In [0]:
df_cat = spark.read.format('csv')\
              .option("inferSchema", "true")\
              .option("header", "true")\
              .load('abfss://bronze@awdedatalake.dfs.core.windows.net/AdventureWorks_Product_Categories')

In [0]:
df_product = spark.read.format('csv')\
              .option("inferSchema", "true")\
              .option("header", "true")\
              .load('abfss://bronze@awdedatalake.dfs.core.windows.net/AdventureWorks_Products')

In [0]:
df_ret = spark.read.format('csv')\
              .option("inferSchema", "true")\
              .option("header", "true")\
              .load('abfss://bronze@awdedatalake.dfs.core.windows.net/AdventureWorks_Returns')

In [0]:
df_sales15 = spark.read.format('csv')\
              .option("inferSchema", "true")\
              .option("header", "true")\
              .load('abfss://bronze@awdedatalake.dfs.core.windows.net/AdventureWorks_Sales_2015')

In [0]:
df_sales16 = spark.read.format('csv')\
              .option("inferSchema", "true")\
              .option("header", "true")\
              .load('abfss://bronze@awdedatalake.dfs.core.windows.net/AdventureWorks_Sales_2016')

In [0]:
df_sales17 = spark.read.format('csv')\
              .option("inferSchema", "true")\
              .option("header", "true")\
              .load('abfss://bronze@awdedatalake.dfs.core.windows.net/AdventureWorks_Sales_2017')

In [0]:
df_sales = spark.read.format('csv')\
              .option("inferSchema", "true")\
              .option("header", "true")\
              .load('abfss://bronze@awdedatalake.dfs.core.windows.net/AdventureWorks_Sales*')

In [0]:
df_terr = spark.read.format('csv')\
              .option("inferSchema", "true")\
              .option("header", "true")\
              .load('abfss://bronze@awdedatalake.dfs.core.windows.net/AdventureWorks_Territories')

In [0]:
df_subcat = spark.read.format('csv')\
              .option("inferSchema", "true")\
              .option("header", "true")\
              .load('abfss://bronze@awdedatalake.dfs.core.windows.net/Product_Subcategories')

#### Transformations

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

##### Calendar Data

In [0]:
# Transformation 1 - Adding new cols to the Calendar df: Year, Quarter, Month, Week, Day

df_cal = df_cal.withColumn('Year', year(col('Date')))\
               .withColumn('Quarter', quarter(col('Date')))\
               .withColumn('Month', month(col('Date')))\
               .withColumn('Week', weekofyear(col('Date')))\
               .withColumn('Day', dayofmonth(col('Date')))

# df_cal.display()

df_cal.write.format('parquet')\
            .mode('overwrite')\
            .option('path', 'abfss://silver@awdedatalake.dfs.core.windows.net/AdventureWorks_Calendar')\
            .save()

##### Customer Data

In [0]:
# Transformation 2 - Adding new cols to the Customer df: FullName

# df_cust = df_cust.withColumn('FullName', concat(col('Prefix'), lit(' '), col('FirstName'), lit(' '), col('LastName'))).display()

df_cust = df_cust.withColumn('FullName', concat_ws(' ', col('Prefix'), col('FirstName'), col('LastName')))

df_cust.write.format('parquet')\
            .mode('overwrite')\
            .option('path', 'abfss://silver@awdedatalake.dfs.core.windows.net/AdventureWorks_Customer')\
            .save()

##### Product_Category Data

In [0]:
df_cat = df_cat.write.format('parquet')\
                    .mode('overwrite')\
                    .option('path', 'abfss://silver@awdedatalake.dfs.core.windows.net/AdventureWorks_Product_Categories')\
                    .save()

##### Products Data

In [0]:
# Transformation 3 - Updating the Product df: ProductSKU, ProductName

df_product = df_product.withColumn('ProductSKU', split(col('ProductSKU'), '-')[0])\
                    .withColumn('ProductName', split(col('ProductName'),' ')[0])


df_product.write.format('parquet')\
                .mode('overwrite')\
                .option('path', 'abfss://silver@awdedatalake.dfs.core.windows.net/AdventureWorks_Products')\
                .save()

##### Returns Data

In [0]:
df_ret.write.format('parquet')\
                .mode('overwrite')\
                .option('path', 'abfss://silver@awdedatalake.dfs.core.windows.net/AdventureWorks_Returns')\
                .save()

##### Territories Data

In [0]:
df_terr.write.format('parquet')\
                .mode('overwrite')\
                .option('path', 'abfss://silver@awdedatalake.dfs.core.windows.net/AdventureWorks_Territories')\
                .save()

##### Product_Subcategories Data

In [0]:
df_subcat = df_subcat.write.format('parquet')\
                    .mode('overwrite')\
                    .option('path', 'abfss://silver@awdedatalake.dfs.core.windows.net/Product_Subcategories')\
                    .save()

##### Sales Data

In [0]:
# Transformation 4 - Updating the Sales df: Update StockDate to TS, Replace S with T for OrderNumber col, New Col - Multiply OrderLineItem and OrderQuantity

df_sales = df_sales.withColumn('StockDate', to_timestamp(col('StockDate')))\
                   .withColumn('OrderNumber', regexp_replace(col('OrderNumber'), 'S', 'T'))\
                   .withColumn('Multiply', col('OrderLineItem') * col('OrderQuantity'))

df_sales.write.format('parquet')\
            .mode('overwrite')\
            .option('path', 'abfss://silver@awdedatalake.dfs.core.windows.net/AdventureWorks_Sales')\
            .save()


##### Sales Analysis

In [0]:
# To get the no of orders received every day and visualize the result

df_sales.groupBy('OrderDate').agg(count('OrderNumber').alias('total_orders_per_day')).display()

OrderDate,total_orders_per_day
2017-01-06,151
2017-01-27,142
2017-02-26,119
2017-01-24,173
2017-06-29,172
2017-02-16,124
2017-04-09,140
2017-02-28,162
2017-03-28,149
2017-06-30,136


Databricks visualization. Run in Databricks to view.

In [0]:
# To fetch and visualize different categories and no of orders belonging to those categories
# df_subcat.display()
df_sales.alias('s')\
        .join(df_product.alias('p'), on = col('s.ProductKey') == col('p.ProductKey'), how = 'left')\
        .join(df_subcat.alias('sc'), on = col('sc.ProductSubcategoryKey') == col('p.ProductSubcategoryKey'), how = 'left')\
        .join(df_cat.alias('c'), on = col('c.ProductCategoryKey') == col('sc.ProductCategoryKey'), how = 'left')\
        .groupBy(col('c.CategoryName'))\
        .agg(count(col('OrderNumber')).alias('total_orders_per_category'))\
        .display()

CategoryName,total_orders_per_category
Bikes,13929
Clothing,8510
Accessories,33607


Databricks visualization. Run in Databricks to view.

In [0]:
# To get the no of orders placed per region, country and continent

df_sales.alias('s')\
        .join(df_terr.alias('t'), on = col('s.TerritoryKey') == col('t.SalesTerritoryKey'), how = 'left')\
        .groupBy(col('t.Continent'), col('t.Country'), col('t.Region'))\
        .agg(count(col('OrderNumber')).alias('total_orders_per_region'))\
        .display()
    

Continent,Country,Region,total_orders_per_region
North America,United States,Central,20
Europe,United Kingdom,United Kingdom,6423
North America,United States,Northwest,8267
North America,United States,Southeast,34
Pacific,Australia,Australia,12409
North America,United States,Northeast,27
North America,United States,Southwest,11463
North America,Canada,Canada,6875
Europe,France,France,5239
Europe,Germany,Germany,5289
